# Python for Medicine: The First Dose

Welcome to Project MedVision's crash course in Python. Every concept you practise today keeps our fictional patient safe on the ward.

## Morning Rounds Briefing

**Patient:** Jordan Nguyen (54, they/them) admitted with community-acquired pneumonia complicated by sepsis.\n**Today's Role:** Junior doctor on the MedVision team preparing the morning handover.

**Learning objectives**
- Track key clinical facts with variables and formatted strings.
- Organise observations using lists and dictionaries that mirror the chart.
- Automate triage decisions with loops, conditionals, and helper utilities.
- Assemble a concise handover draft that flows into the AI lab in Notebook 02.

Plan for ~60 minutes of focused practice.

### How to Use This Notebook
- Run each cell from top to bottom. Restart (`Runtime → Restart and run all`) if you get lost.
- Challenge cells raise `NotImplementedError` until you complete them—this is intentional so mistakes surface immediately.
- Solutions live in collapsed cells. Only open them after giving the exercise an honest try.
- Need a fresh start? Rerun the setup cell to rebuild the environment.

## 0. Setup: Prep Your Colab Cart

This cell downloads the MedVision toolkit and installs every dependency.
Run it once per fresh Colab session. If anything fails, the cell stops immediately so you can fix the issue.

In [ ]:
from __future__ import annotations

import subprocess
import sys
from pathlib import Path
from typing import Final

REPO_URL: Final[str] = "https://github.com/your-repo/python-for-medicine-bootcamp.git"

workspace = Path.cwd()

# Try to find the project root
# Case 1: Running from project root (workspace contains src/medvision_toolkit)
# Case 2: Running from notebooks/ subdirectory (parent contains src/medvision_toolkit)
# Case 3: Running in Colab (need to clone repository)

if (workspace / "src" / "medvision_toolkit").exists():
    repo_root = workspace
    print("✓ Detected Project MedVision source in the current workspace.")
elif (workspace.parent / "src" / "medvision_toolkit").exists():
    repo_root = workspace.parent
    print("✓ Detected Project MedVision source in parent directory.")
    print(f"  Project root: {repo_root}")
else:
    # Colab or git clone scenario
    repo_root = workspace / "python-for-medicine-bootcamp"
    if repo_root.exists():
        print("Existing checkout found. Pulling latest changes...")
        result = subprocess.run(
            ["git", "-C", str(repo_root), "pull", "--ff-only"],
            capture_output=True,
            text=True,
        )
        if result.returncode != 0:
            print(result.stdout)
            print(result.stderr, file=sys.stderr)
            raise RuntimeError("git pull failed; please resolve and rerun the cell.")
    else:
        print(f"Cloning repository from {REPO_URL} …")
        result = subprocess.run(
            ["git", "clone", "--depth", "1", REPO_URL, str(repo_root)],
            capture_output=True,
            text=True,
        )
        if result.returncode != 0:
            print(result.stdout)
            print(result.stderr, file=sys.stderr)
            raise RuntimeError("git clone failed; check the URL or your connection.")

# Install dependencies (only in Colab or if explicitly needed)
requirements_path = repo_root / "requirements.txt"
if not requirements_path.exists():
    raise FileNotFoundError(f"Could not locate requirements.txt at {requirements_path}")

# Check if we're in Colab (has google.colab module)
try:
    import google.colab
    in_colab = True
except ImportError:
    in_colab = False

if in_colab:
    print("Installing Python dependencies (this may take a minute)…")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-r", str(requirements_path)])
else:
    print("Running locally. Assuming dependencies are already installed.")
    print("If you encounter import errors, run: pip install -r requirements.txt")

# Add src directory to path
project_src = repo_root / "src"
if str(project_src) not in sys.path:
    sys.path.insert(0, str(project_src))
    print(f"✓ Added {project_src} to Python path")

print("✅ Environment ready. Proceed to the next cell.")

### Quick Systems Check
If everything installed correctly, the toolkit will summarise our patient without errors.

In [ ]:
from medvision_toolkit.learning.patient_profiles import load_sample_patient, summarize_patient

patient_profile = load_sample_patient()
print(summarize_patient(patient_profile))


## 1. Patient Intake: Variables & Strings

Variables are our labels for important clinical facts—think room assignments, attending physicians, or today’s rounding date.

In [ ]:
from datetime import date

round_date = date.today()
attending_on_service = patient_profile['demographics']['attending']
room_assignment = '5B-12'
clinical_focus = 'Respiratory status and mobility'

print(f"Rounds Date: {round_date:%B %d, %Y}")
print(f"Attending on service: {attending_on_service}")
print(f"Room assignment: {room_assignment}")
print(f"Today's clinical focus: {clinical_focus}")


✨ **Clinical side note**: Clear variable names mirror clear charting. When your future self reads this code, they should immediately know which clinical detail they are looking at.

### Your Turn: Challenge 1 — Body Composition Check
1. Store Jordan's height (metres) and weight (kg) in variables.
2. Use `calculate_bmi` from `medvision_toolkit` to compute the BMI.
3. Print a message such as `BMI 25.7 — monitor nutritional status`.
4. Decide on the guidance message using an `if/else` block.

In [ ]:
# TODO: Replace the placeholder below with your solution.
raise NotImplementedError('Challenge 1: calculate BMI and print a guidance message.')


In [ ]:
from medvision_toolkit.learning.patient_profiles import calculate_bmi

height_m = 1.65
weight_kg = 72.0
patient_bmi = calculate_bmi(weight_kg=weight_kg, height_m=height_m)

if patient_bmi >= 25:
    guidance = 'BMI elevated — continue dietician consult.'
else:
    guidance = 'BMI within goal range — maintain current plan.'

print(f"BMI {patient_bmi} — {guidance}")


Great! Variables now capture the snapshot facts we need for rounds.

## 2. Trend Tracking with Lists

Lists help us follow a value over time—perfect for vitals, labs, or pain scores.

In [ ]:
patient_profile = load_sample_patient()  # fresh copy to avoid accidental edits

bp_trend = patient_profile['vitals']['systolic_bp']
pain_trend = patient_profile['vitals']['pain_scores']

print(f"Systolic BP readings: {bp_trend}")
print(f"Max BP this week: {max(bp_trend)} mmHg")
print(f"Min BP this week: {min(bp_trend)} mmHg")

print(f"Pain scores: {pain_trend}")
print(f"Number of pain observations: {len(pain_trend)}")
print(f"Highest pain score: {max(pain_trend)}")


Try describing what these trends mean clinically. Do you expect any interventions today?

In [ ]:
from medvision_toolkit.learning.patient_profiles import plot_vital_trends

plot_vital_trends(patient_profile['vitals'])


### Your Turn: Challenge 2 — Convert and Summarise
1. Create a new list called `bp_kpa` converting each systolic BP to kilopascals (`mmHg * 0.133`).
2. Print the converted list.
3. Calculate the average pain score and print a short interpretation (e.g., `Average pain 4.3 — improving`).

In [ ]:
# TODO: Replace the placeholder below with your solution.
raise NotImplementedError('Challenge 2: work with lists and basic statistics.')


In [ ]:
conversion_factor = 0.133
bp_kpa = [round(reading * conversion_factor, 1) for reading in bp_trend]
print(f"Systolic BP (kPa): {bp_kpa}")

average_pain = sum(pain_trend) / len(pain_trend)
if average_pain >= 6:
    interpretation = 'Pain poorly controlled — discuss with attending.'
elif average_pain >= 3:
    interpretation = 'Pain improving — continue current regimen.'
else:
    interpretation = 'Pain well controlled.'

print(f"Average pain {average_pain:.1f} — {interpretation}")


## 3. Patient Charts as Dictionaries

Dictionaries (and the `TypedDict` schema defined in our toolkit) mirror structured chart sections: demographics, vitals, consults, and more.

In [ ]:
patient_profile = load_sample_patient()

print('Top-level keys:', list(patient_profile.keys()))
print('Demographics:')
for key, value in patient_profile['demographics'].items():
    print(f"  {key}: {value}")


Notice how the structure matches what you would expect in an electronic health record.

### Your Turn: Challenge 3 — Consult Coordination
1. Add `'Nutrition services'` to the `consults` list if it is not already present.
2. Create a new key `consult_note` that summarises why nutrition is involved.
3. Print the consult note to confirm.

In [ ]:
# TODO: Replace the placeholder below with your solution.
raise NotImplementedError('Challenge 3: update the consult list and add a note.')


In [ ]:
if 'Nutrition services' not in patient_profile['consults']:
    patient_profile['consults'].append('Nutrition services')

patient_profile['consult_note'] = (
    'Consult Nutrition to support high-protein meals and monitor intake while appetite returns.'
)
print(patient_profile['consult_note'])


## 4. Loops & Conditionals: Rapid Response Dashboards

Loops let us audit every reading, while conditionals decide which ones need escalation.

In [ ]:
from medvision_toolkit.learning.patient_profiles import triage_pain_scores

patient_profile = load_sample_patient()
for message in triage_pain_scores(patient_profile['vitals']['pain_scores']):
    print(message)


### Your Turn: Challenge 4 — Blood Pressure Watchlist
1. Iterate through the systolic BP readings.
2. Build a list called `bp_alerts` with one message per reading:
   - `Hypertension alert: {bp} mmHg` when `bp >= 140`
   - `Consider step-down monitoring: {bp} mmHg` when `bp <= 120`
   - Otherwise `Within goal range: {bp} mmHg`
3. Print each alert.

In [ ]:
# TODO: Replace the placeholder below with your solution.
raise NotImplementedError('Challenge 4: build a BP alert list with a for loop.')


In [ ]:
bp_alerts = []
for reading in patient_profile['vitals']['systolic_bp']:
    if reading >= 140:
        bp_alerts.append(f"Hypertension alert: {reading} mmHg")
    elif reading <= 120:
        bp_alerts.append(f"Consider step-down monitoring: {reading} mmHg")
    else:
        bp_alerts.append(f"Within goal range: {reading} mmHg")

for alert in bp_alerts:
    print(alert)


## 5. Functions as Clinical Protocols

Functions bundle repeatable logic—just like a standard order set. We wrote them once in the toolkit so you can call them here without wrestling with implementation details.

In [ ]:
from medvision_toolkit.learning.patient_profiles import compute_vital_statistics, generate_handover_note

patient_profile = load_sample_patient()
stats = compute_vital_statistics(patient_profile['vitals'])
print(stats)

handover_note = generate_handover_note(
    patient_profile,
    action_items=patient_profile['outstanding_tasks'],
)
print("\n" + handover_note)

In [ ]:
from medvision_toolkit.learning.patient_profiles import PatientProfile

# TODO: Complete the function and print the resulting note.
def draft_handover_for_night_team(profile: PatientProfile) -> str:
    raise NotImplementedError('Capstone: combine business rules with generate_handover_note.')

print(draft_handover_for_night_team(patient_profile))


In [ ]:
from medvision_toolkit.learning.patient_profiles import (
    compute_vital_statistics,
    generate_handover_note,
)


def draft_handover_for_night_team(profile: PatientProfile) -> str:
    vitals = profile['vitals']
    stats = compute_vital_statistics(vitals)
    action_items = list(profile['outstanding_tasks'])

    if stats.latest_systolic_bp >= 140:
        action_items.append('Recheck blood pressure in 1 hour.')

    if vitals['pain_scores'][-1] >= 4:
        action_items.append('Offer guided breathing before bedtime.')

    return generate_handover_note(profile, action_items)


print(draft_handover_for_night_team(patient_profile))


## 6. Charting to AI Pipeline

- You captured patient context with variables, lists, and dictionaries.
- You reasoned about alerts using loops and guardrails.
- You generated a handover note via a typed helper function.

In Notebook 02 you'll hand this summary to `RadiologyAI`, our vision-language model. The structured data you curated here becomes the context that the AI uses to justify its image findings.

### Next Steps
1. Save a copy of your handover note for tomorrow's lab.
2. Open `02_project_medvision_lab.ipynb` to connect your intake logic to real imaging AI.
3. Celebrate—you just completed your first clinical coding rounds!